In [1]:
import pandas as pd
import numpy as np

# Import and clean data

In [2]:
path = "Datasets/"
train_pos_path = path + "train_pos.txt"
train_neg_path = path + "train_neg.txt"
test_path = path + "test_data.txt"

Import training positive and negative:

In [3]:
train_pos_df = pd.read_csv(train_pos_path, delimiter="\\n", header=None, names = ["tweets"], engine='python')
# remove the <user> tag from all the tweets
train_pos_df["tweets"] = train_pos_df["tweets"].str.replace("<user>", "")
# put the prediction value (1 for positive)
train_pos_df["prediction"] = 1

train_neg_df = pd.read_csv(train_neg_path, delimiter="\\n", header=None, names = ["tweets"], engine='python')
# remove the <user> tag from all the tweets
train_neg_df["tweets"] = train_neg_df["tweets"].str.replace("<user>", "")
# put the prediction value (-1 for negative)
train_neg_df["prediction"] = -1

Concatenate both of the dataset to have a unique train set:

In [4]:
# concatenate train positive and negative
to_concat = [train_pos_df, train_neg_df]
df_train = pd.concat(to_concat)

# remove duplicates 
# df_train[df_train["tweets"] == " thanks love !"]
df_train = df_train.drop_duplicates(subset=["tweets"])
# reset indexes
df_train = df_train.reset_index(drop=True)

Split randomly the train set into train/set (80%-20%) in order to evaluate performance:

In [5]:
msk = np.random.rand(len(df_train)) < 0.8

train = df_train[msk]
test = df_train[~msk]

In [6]:
positive_in_train = len(train[train["prediction"] == 1])
negative_in_train = len(train[train["prediction"] == -1])

positive_in_test =len(test[test["prediction"] == 1])
negative_in_test = len(test[test["prediction"] == -1])

In [7]:
print("There is {} positive, {} negative tweets in train set ".format(positive_in_train, negative_in_train))

print("There is {} positive, {} negative tweets in test set ".format(positive_in_test, negative_in_test))



There is 72183 positive, 72854 negative tweets in train set 
There is 18050 positive, 18220 negative tweets in test set 


Import the test set: (TBD)

In [8]:
df_test = pd.read_csv(test_path, sep='^([^,]+),', engine='python', header=None, names = ["index", "tweets"]).reset_index(drop=True)
# remove the <user> tag from all the tweets
df_test["tweets"] = df_test["tweets"].str.replace("<user>", "")

Prepare for BERT the dataframes: [THE TUTO MAGIQUE](https://medium.com/swlh/a-simple-guide-on-using-bert-for-text-classification-bbf041ac8d04)

In [9]:
train_df_bert = pd.DataFrame({
    'id':train.index,
    'label':train["prediction"],
    'alpha':['a']*train.shape[0],
    'text': train["tweets"].replace(r'\n', ' ', regex=True)
})

train_df_bert.head()

,id,label,alpha,text
0,0,1,a,i dunno justin read my mention or not . only ...
1,1,1,a,"because your logic is so dumb , i won't even c..."
2,2,1,a,""" just put casper in a box ! "" looved the bat..."
3,3,1,a,thanks sir > > don't trip lil mama ... just ...
4,4,1,a,visiting my brother tmr is the bestest birthda...


In [10]:
dev_df_bert = pd.DataFrame({
    'id':test.index,
    'label':test["prediction"],
    'alpha':['a']*test.shape[0],
    'text': test["tweets"].replace(r'\n', ' ', regex=True)
})

dev_df_bert.head()

,id,label,alpha,text
9,9,1,a,this is were i belong
13,13,1,a,agreed ! 12 more days left tho
37,37,1,a,apparently not )
47,47,1,a,wait my brother has one ! let's get a picture
55,55,1,a,"well mr . schmidt / knight , aren't you the c..."


In [11]:
test_df_bert = pd.DataFrame({
    'id':df_test["index"],
    'alpha':['a']*df_test.shape[0],
    'text': df_test["tweets"].replace(r'\n', ' ', regex=True)
})

In [12]:
train_df_bert.to_csv(path + 'train.tsv', sep='\t', index=False, header=False)
dev_df_bert.to_csv(path + 'dev.tsv', sep='\t', index=False, header=False)
test_df_bert.to_csv(path + 'test.tsv', sep='\t', index=False, header=False)

In [13]:
def get_labels(self):
    return [-1, 1]


# https://mc.ai/how-to-do-text-binary-classification-with-bert/

In [14]:
!pip install bert-tensorflow
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import pickle
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

In [15]:
def pretty_print(result):
    df = pd.DataFrame([result]).T
    df.columns = ["values"]
    return df

In [16]:
def create_tokenizer_from_hub_module(bert_model_hub):
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(bert_model_hub)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

def make_features(dataset, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN):
    input_example = dataset.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)
    features = bert.run_classifier.convert_examples_to_features(input_example, label_list, MAX_SEQ_LENGTH, tokenizer)
    return features

def create_model(bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      bert_model_hub,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.compat.v1.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.compat.v1.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)

# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(bert_model_hub, num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        bert_model_hub, is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn

def estimator_builder(bert_model_hub, OUTPUT_DIR, SAVE_SUMMARY_STEPS, SAVE_CHECKPOINTS_STEPS, label_list, LEARNING_RATE, num_train_steps, num_warmup_steps, BATCH_SIZE):

    # Specify outpit directory and number of checkpoint steps to save
    run_config = tf.estimator.RunConfig(
        model_dir=OUTPUT_DIR,
        save_summary_steps=SAVE_SUMMARY_STEPS,
        save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

    model_fn = model_fn_builder(
      bert_model_hub = bert_model_hub,
      num_labels=len(label_list),
      learning_rate=LEARNING_RATE,
      num_train_steps=num_train_steps,
      num_warmup_steps=num_warmup_steps)

    estimator = tf.estimator.Estimator(
      model_fn=model_fn,
      config=run_config,
      params={"batch_size": BATCH_SIZE})
    return estimator, model_fn, run_config


In [25]:
def run_on_dfs(train, test, DATA_COLUMN, LABEL_COLUMN, 
               MAX_SEQ_LENGTH = 128,
              BATCH_SIZE = 2,
              LEARNING_RATE = 2e-5,
              NUM_TRAIN_EPOCHS = 3.0,
              WARMUP_PROPORTION = 0.1,
              SAVE_SUMMARY_STEPS = 100,
               SAVE_CHECKPOINTS_STEPS = 10000,
              bert_model_hub = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"):

    label_list = train[LABEL_COLUMN].unique().tolist()
    
    tokenizer = create_tokenizer_from_hub_module(bert_model_hub)

    train_features = make_features(train, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)
    test_features = make_features(test, label_list, MAX_SEQ_LENGTH, tokenizer, DATA_COLUMN, LABEL_COLUMN)

    num_train_steps = 101 ### int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
    num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

    estimator, model_fn, run_config = estimator_builder(
                                  bert_model_hub, 
                                  OUTPUT_DIR, 
                                  SAVE_SUMMARY_STEPS, 
                                  SAVE_CHECKPOINTS_STEPS, 
                                  label_list, 
                                  LEARNING_RATE, 
                                  num_train_steps, 
                                  num_warmup_steps, 
                                  BATCH_SIZE)

    train_input_fn = bert.run_classifier.input_fn_builder(
        features=train_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=True,
        drop_remainder=False)

    estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)

    test_input_fn = run_classifier.input_fn_builder(
        features=test_features,
        seq_length=MAX_SEQ_LENGTH,
        is_training=False,
        drop_remainder=False)

    result_dict = estimator.evaluate(input_fn=test_input_fn, steps=None)
    return result_dict, estimator

In [18]:
import random
random.seed(10)

In [19]:
OUTPUT_DIR = 'output'

# Data to features

In [21]:
myparam = {
        "DATA_COLUMN": "tweets",
        "LABEL_COLUMN": "prediction",
        "LEARNING_RATE": 2e-5,
        "NUM_TRAIN_EPOCHS":10,
"bert_model_hub":"https://tfhub.dev/google/bert_multi_cased_L-12_H-768_A-12/1"
    }
    

In [22]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.INFO)


In [23]:
result, estimator = run_on_dfs(train, test, **myparam)


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 145037


INFO:tensorflow:Writing example 0 of 145037


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:global_step/sec: 0.34752


INFO:tensorflow:global_step/sec: 0.34752


INFO:tensorflow:loss = 0.00342097, step = 2101 (287.753 sec)


INFO:tensorflow:loss = 0.00342097, step = 2101 (287.753 sec)


KeyboardInterrupt: 

In [26]:
result, estimator = run_on_dfs(train, test, **myparam)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Writing example 0 of 145037


INFO:tensorflow:Writing example 0 of 145037


INFO:tensorflow:*** Example ***


...

INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Running local_init_op.


INFO:tensorflow:Done running local_init_op.


INFO:tensorflow:Done running local_init_op.


KeyboardInterrupt: 

In [ ]:
result, estimator = run_on_dfs(train, test, **myparam)

In [ ]:
pretty_print(result)

In [ ]:
import pickle

with open("estimator.txt", 'wb') as f:
    pickle.dump((estimator), f)

In [ ]:
# https://colab.research.google.com/drive/1TUxZcKaMbGh1OV2-3fwv_FPu2t9-88tS#scrollTo=YLOvtveTMqwp